In [ ]:
from os import listdir
from os.path import isfile,join
from PIL import Image
import numpy as np
import pickle
import cv2
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import math
print(cv2.__version__)

In [ ]:
#labels Train
tr_label_file = open("/Volumes/MARCOS E/PAN2018/train/es.txt")
labels_b = [author.split(":::") for author in tr_label_file.read().split("\n")]
labels = {}
for author in labels_b:
    if len(author) > 1:
        labels[author[0]] = author[1]
print("OK")

In [ ]:
#labels Test
te_label_file = open("/Volumes/MARCOS E/PAN2018/test/es.txt")
labels_b = [author.split(":::") for author in te_label_file.read().split("\n")]
labels_test = {}
for author in labels_b:
    if len(author) > 1:
        labels_test[author[0]] = author[1]
print("OK")


In [ ]:
def load_caffe_models(prototxt,caffemodel):
    """
    Carga de un modelo de redes neuronales profundas (Caffe) 
    Necesita la arquitectura de la red y los pesos de las neuronas
    """
    net =cv2.dnn.readNetFromCaffe(prototxt,caffemodel)
    return net

In [ ]:
#Carga de un modelo HaarCascade pre-entrenado para detección de caras
detector_caras = cv2.CascadeClassifier("frontdetector.xml") 
gender_net = load_caffe_models("deploy_gender.prototxt","gender_net.caffemodel") #Detección de género con Caffe 
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
gender_list = ['Male', 'Female']

In [ ]:
def calculaPorcentaje(dirname,labels):
    directorios = [f for f in listdir(dirname)]
    globalCorpus = []
    globalLabels = []
    imagenes_erroneas = 0
    imagenes_totales = 0
    #Recorremos los directorios
    for directorio in directorios:
        #Obtenemos la etiqueta asociada (Hombre o mujer)
        etiqueta = labels.get(directorio)
        corpusLocal = []
        if  etiqueta != None:
            imagenes = [f for f in listdir(dirname+"/"+directorio)] #Obtenemos las 10 imágenes por autor
            imagenes_correctas = 0
            #Recorremos cada imagen
            for imagen in imagenes:
                imagenes_totales+=1
                try:
                    image = cv2.imread(dirname+"/"+directorio+"/"+imagen) 
                    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) #Transformamos la imagen a Blanco y negro
                    #Aplicamos HaarCascade --> lista de caras
                    caras = detector_caras.detectMultiScale(gray,1.5,5)
                    if len(caras)>0:
                        #Recorremos cada cara obteniendola para lanzarla contra la red neuronal
                        for (x,y,w,h) in caras:
                            face_img = image[y:y+h, x:x+w].copy()
                            #Lanzamos la cara contra la red neuronal...
                            blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
                            gender_net.setInput(blob)
                            gender_preds = gender_net.forward()
                            #Obtenemos la clase más probable
                            gender = gender_list[gender_preds[0].argmax()]
                            corpusLocal.append(-1 if gender == "Male" else 1)
                    imagenes_correctas+=1
                except:
                    imagenes_erroneas+=1
            hombres = sum([1 for element in corpusLocal if element ==-1]) #número de hombres por autor
            #porcentaje de hombre en las fotos del autor
            globalCorpus.append(hombres/len(corpusLocal) if len(corpusLocal) > 0 else 0) 
            globalLabels.append(etiqueta)
    print("Imagenes Erroneas: %d" %imagenes_erroneas)         
    print(len(globalLabels) ==len(globalCorpus))
    return globalCorpus, globalLabels



In [ ]:
globalCorpusTrain, globalLabelsTrain = calculaPorcentaje("/Volumes/MARCOS E/PAN2018/train/photo",labels)
globalCorpusTest, globalLabelsTest = calculaPorcentaje("/Volumes/MARCOS E/PAN2018/test/es/photo",labels_test)

In [ ]:
def saveObject(outputFile, representacion,etiquetas):
    """
    Guardo la representación y las etiquetas ya que es muy costoso calcularlo
    """
    with open(outputFile,"wb") as fh:
        object = representacion, etiquetas
        pickle.dump(object,fh)

In [ ]:
saveObject("pcarasTrain",globalCorpusTrain,globalLabelsTrain)
saveObject("pcarasTest",globalCorpusTest,globalLabelsTest)

In [ ]:
def openfile(fileName):
    """
    Dado el nombre de un fichero devuelve la representación y las etiquetas
    """
    with open(fileName,"rb") as fh:
        representacion,etiquetas = pickle.load(fh)
    return etiquetas

In [ ]:
#Cargamos los datos
globalCorpusTrain, globalLabelsTrain = openfile("pcarasTrain")
globalCorpusTest, globalLabelsTest = openfile("pcarasTest")

In [ ]:
def intervalo95(p,datos):
    """
    Calcula el intervalo al 95% 
    :param p: probabilidad de acierto
    :param datos: numero de datos del conjunto de test
    """
    e = 1.96*math.sqrt((p*(1-p))/datos)
    s = "[%.3f , %.3f]" % (p-e,p+e)
    return s

In [ ]:
def cross_validation(clasificador,xtrain,xlabels,bloques):
    scores = cross_val_score(clasificador, xtrain, xlabels, cv=bloques, scoring='f1_macro')
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    return scores.mean()

In [ ]:
cs = [1,10,100,1000,10000]
kernel= ["linear","rbf"]
res = "c \t \t k \t \t Accuracy \t \t Inter95 \n"
res += "---------------------SVM--------------------------\n"
for k in kernel:
    for c in cs:
        clf = svm.SVC(C = c,kernel=k)
        media = cross_validation(clf,np.array(globalCorpusTrain).reshape((3000,1)),globalLabelsTrain,10)
        clf.fit(np.array(globalCorpusTrain).reshape((3000,1)),globalLabelsTrain)
        p = clf.score(np.array(globalCorpusTest).reshape((2200,1)),globalLabelsTest)
        aux = "%d \t \t  %s \t \t %.4f \t \t %s \n" % (c,k[0:3],p, intervalo95(p,len(globalLabelsTest)))
        print(aux)
        res+= aux
print(res)

In [ ]:
cs = [1,10,100,1000,10000,100000]
res = "c \t \t Accuracy \t \t Inter95 \n"
res += "---------------------Logistic--------------------------\n"
for c in cs:
    clf = LogisticRegression(C=c)
    media = cross_validation(clf,np.array(globalCorpusTrain).reshape((3000,1)),globalLabelsTrain,10) 
    clf.fit(np.array(globalCorpusTrain).reshape((3000,1)),globalLabelsTrain)
    p = clf.score(np.array(globalCorpusTest).reshape((2200,1)),globalLabelsTest)
    aux = "%d \t \t %.3f \t \t %s \n" %(c,p,intervalo95(p,len(globalLabelsTest)))
    print(aux)
    res+= aux
print(res)